In [1]:
import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import datetime
import pickle
import copy

import os
import rasterio
import shapely.geometry

#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

In [2]:
from sklearn.cluster import dbscan

In [3]:
output_dir = "/home/ubuntu/data/TX_paired/"

In [4]:
#first time use:
geo_df = pickle.load( open( output_dir+"GeoDataFrame.pickled", "rb" ))

#after use:
#geo_df = pickle.load( open( output_dir+"GeoDataFrame_fine.pickled", "rb" ))
geo_df['DBScan']=None
geo_df.set_index("tile_no")
geo_df.head(1)

,tile_no,%flooded,post-storm_full,pre-storm_full,post-storm_resized,pre-storm_resized,course_mask_full,course_mask_resized,fine_make_filename,footprint,dry/wet,mask_poly,tile_transform,geometry,DBScan
0,0,0.80323,0_post_resize_img,0_pre_resize_img,0_post_full_img,0_pre_full_img,0_mask,0_resize_mask,None,3002220.tif,wet,(POLYGON ((-95.57181511210993 29.4410615808823...,"[222822.4, 0.0, 0.0, -222822.4, 21295616.0, 65...",POLYGON ((-95.56985294117646 29.44106158088235...,None


In [5]:
#get tile_no for those tiles with more than a little flooding
t = geo_df["%flooded"] > 0.00
sum(t)
tiles = geo_df[t].tile_no

In [6]:
tiles.values

array([0, 1, 2, ..., 4402, 4403, 4404], dtype=object)

In [7]:
#run a subset of data at a time in case DBSCAN kills the kernel
for tile_no in tiles.values[:]:
    print("working on tile #:",tile_no)
    
    #load files
    img_post = np.load(output_dir+'%d_post_resize_img.npy'%tile_no)
    img_pre  = np.load(output_dir+'%d_pre_resize_img.npy'%tile_no)
    mask = np.load(output_dir+'%d_resize_mask.npy'%tile_no)
    
    #combine features
    features=img_post
    #add in subtracted image is a feature
    #img_diff = 0.2*(img_post-img_pre)
    #features = np.stack((img_post[:,:,0],img_post[:,:,1],img_post[:,:,2],img_diff[:,:,0],img_diff[:,:,1],img_diff[:,:,2]),axis=2)
    #features.shape
    
    flat_img = np.reshape(features,(features.shape[0]**2,features.shape[2]))
    
    clustered = dbscan(flat_img,eps=1.7,min_samples=100,n_jobs=-2)
    
    #make new mask (of all labels for now)
    side = int(clustered[1].shape[0]**0.5)
    DBScan_mask = np.reshape(clustered[1],(side,side))
    #make backup copy to save for review
    DBScan_mask_original = copy.deepcopy(DBScan_mask)
    
    #order the clusters
    c_count=Counter(DBScan_mask.flatten())
    order = [x[0] for x in c_count.most_common() if x[0]>= 0]  #gets cluster index, AND throws out the negative-1 group

    
    #routine that checks of the masked area is too grey or green, and moves down the list of clusters it is
    
    ready = False
    dry = False
    while ready==False:
        if len(order)==0:
            ready = True
            dry = True
            c_id = None
            break
            #continue

        c_id = order[0]
        color_sum = (img_post*(np.expand_dims(mask==c_id,axis=2))).sum(axis=(0,1))
        #print(color_sum)
        #print(1.0*color_sum[1]/color_sum[0])

        #reject if too grey/white/black 0.108-->0.102-->0.100
        if color_sum.std()*1.0/color_sum.mean() < 0.100:   
            print("too grey, reject cluster",order[0],color_sum.std()*1.0/color_sum.mean())
            order.pop(0)
            continue

        #if the most common color is too green, reject it and move to the next  1.1-->1.2-->1.18--> 1.13-->1.1

        elif 1.0*color_sum[1]/color_sum[0] > 1.2:  
            print(color_sum[1]*1.0/color_sum[0])
            print("too green, reject cluster ",order[0])
            order.pop(0)

        else:ready=True

    print("floodwater/mud at id",c_id)
    
    if dry == True: fine_mask = np.zeros((side,side),dtype='int64')  #the mask data type should match
    else: fine_mask = 1*(DBScan_mask==c_id)
    
    np.save(output_dir+"%d_256_DBSCAN"%tile_no, DBScan_mask_original)
    np.save(output_dir+"%d_256_fine_mask"%tile_no, fine_mask)
        
    #update the entry to the geopandas Dataframe with the filename
    geo_df.DBScan[tile_no] = "%d_256_DBSCAN"%tile_no
    geo_df.fine_make_filename[tile_no] = "%d_256_fine_mask"%tile_no

    #write geopandas to file too
    geo_df.to_pickle(output_dir+"GeoDataFrame_fine.pickled")


('working on tile #:', 0)
[833074 901314 522068]
('floodwater/mud at id', 0)
('working on tile #:', 1)
[2391991 2616671 1516053]
('floodwater/mud at id', 0)
('working on tile #:', 2)
[2209550 2498232 1387768]
('floodwater/mud at id', 0)
('working on tile #:', 3)
[4595520 5356556 4788798]
('too grey, reject cluster', 0, 0.065732797609628302)
[0 0 0]
('floodwater/mud at id', 1)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars


('working on tile #:', 4)
[0 0 0]
('floodwater/mud at id', 2)
('working on tile #:', 5)


KeyboardInterrupt: 

In [ ]:
"""    #plot for monitoring
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(18,6))
    ax1.set_title("Post-flood Image")
    ax2.set_title("Post-flood Image w/ mask")
    ax3.set_title("Pre-flood Image")
    ax4.set_title("clusters")
    ax1.imshow(img_post)
    ax2.imshow(img_post)
    ax2.imshow(fine_mask,cmap='bwr',alpha = 0.2)
    ax3.imshow(img_pre)
    plt.imshow(DBScan_mask)
    plt.colorbar()
    plt.show();"""

# plotting routines

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
ax1.set_title("Flood Water Only")
ax2.set_title("Flood Water removed")
ax1.imshow(img_post*(np.expand_dims(DBScan_mask==c_id,axis=2)) )
ax2.imshow(img_post*(np.expand_dims(DBScan_mask!=c_id,axis=2)));

fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(18,6))
ax1.set_title("difference image")
ax2.set_title("New Image Mask(Label)")
ax3.set_title("DigitalGlobe MDA Shapefile label")
ax1.imshow(img_diff)
ax2.imshow(255.*(DBScan_mask==c_id))
ax3.imshow(mask);

fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(18,6))
ax1.set_title("Post-flood Image")
ax2.set_title("DBSCAN Clusters")
ax3.set_title("Pre-flood Image")
ax1.imshow(img_post)
ax2.imshow(DBScan_mask)
ax3.imshow(img_pre);

fig, (ax1, ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(18,6))
ax1.set_title("Post-flood Image")
ax2.set_title("Post-flood Image w/ mask")
ax3.set_title("Pre-flood Image")
ax4.set_title("clusters")
ax1.imshow(img_post)
ax2.imshow(img_post)
ax2.imshow(1-(DBScan_mask==c_id),cmap='bwr',alpha = 0.2)
ax3.imshow(img_pre)
plt.imshow(DBScan_mask)
plt.colorbar();